<a href="https://colab.research.google.com/github/riddhikaa/amamzon_ml_2024/blob/main/crnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import json
import sys
sys.path.append('/content/drive/MyDrive/cb lects/66e31d6ee96cd_student_resource_3/student_resource 3/src')
sys.path.append('/content/drive/MyDrive/cb lects/66e31d6ee96cd_student_resource_3/student_resource 3')

from src.utils import download_images
from src.constants import entity_unit_map

class ProductImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        self.entity_to_idx = {entity: idx for idx, entity in enumerate(entity_unit_map.keys())}


        all_units = set(unit for units in entity_unit_map.values() for unit in units)

        all_units.add('unknown')

        self.unit_to_idx = {unit: idx for idx, unit in enumerate(all_units)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_name = os.path.join(self.img_dir, f"{row.name}.jpg")

        if os.path.exists(img_name):
            image = Image.open(img_name).convert('RGB')
        else:
            image = Image.new('RGB', (224, 224), color='black')

        if self.transform:
            image = self.transform(image)

        entity_name = row['entity_name']
        entity_idx = self.entity_to_idx[entity_name]

        if not self.is_test:

            value, unit = parse_entity_value(row['entity_value'])


            if unit not in self.unit_to_idx:
                unit = 'unknown'

            unit_idx = self.unit_to_idx[unit]
            return image, entity_idx, value, unit_idx, row.name
        else:
            return image, entity_idx, 0, 0, row.name

def parse_entity_value(entity_value):
    try:
        parts = entity_value.split()
        if parts[0].startswith("["):
            range_values = parts[0].strip('[]').split(',')

            value = np.mean([float(v) for v in range_values if v])
        else:
            value = float(parts[0])
        unit = ' '.join(parts[1:])
        return value, unit
    except Exception as e:
        print(f"Error decoding entity value: {entity_value}, Error: {e}")
        return 0.0, "unknown"

class CRNN(nn.Module):
    def __init__(self, num_entities, num_units):
        super(CRNN, self).__init__()
        resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.cnn = nn.Sequential(*list(resnet.children())[:-2])
        self.rnn = nn.LSTM(2048, 512, bidirectional=True, batch_first=True)
        self.fc_value = nn.Linear(1024, 1)
        self.fc_entity = nn.Linear(1024, num_entities)
        self.fc_unit = nn.Linear(1024, num_units)

    def forward(self, x):
        features = self.cnn(x)
        features = features.permute(0, 2, 3, 1)
        features = features.view(features.size(0), -1, features.size(-1))
        output, _ = self.rnn(features)
        value_output = self.fc_value(output[:, -1, :])
        entity_output = self.fc_entity(output[:, -1, :])
        unit_output = self.fc_unit(output[:, -1, :])
        return value_output, entity_output, unit_output

def train(model, train_loader, criterion_value, criterion_entity, criterion_unit, optimizer, device):
    model.train()
    for images, entity_idxs, values, unit_idxs, _ in tqdm(train_loader):
        images = images.to(device)
        entity_idxs = entity_idxs.to(device)
        values = values.float().to(device)
        unit_idxs = unit_idxs.to(device)

        optimizer.zero_grad()
        value_outputs, entity_outputs, unit_outputs = model(images)
        loss_value = criterion_value(value_outputs.squeeze(), values)
        loss_entity = criterion_entity(entity_outputs, entity_idxs)
        loss_unit = criterion_unit(unit_outputs, unit_idxs)
        loss = loss_value + loss_entity + loss_unit
        loss.backward()
        optimizer.step()

def validate(model, val_loader, criterion_value, criterion_entity, criterion_unit, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, entity_idxs, values, unit_idxs, _ in tqdm(val_loader):
            images = images.to(device)
            entity_idxs = entity_idxs.to(device)
            values = values.float().to(device)
            unit_idxs = unit_idxs.to(device)

            value_outputs, entity_outputs, unit_outputs = model(images)
            loss_value = criterion_value(value_outputs.squeeze(), values)
            loss_entity = criterion_entity(entity_outputs, entity_idxs)
            loss_unit = criterion_unit(unit_outputs, unit_idxs)
            loss = loss_value + loss_entity + loss_unit
            total_loss += loss.item()
    return total_loss / len(val_loader)

def predict(model, test_loader, device, idx_to_entity, idx_to_unit):
    model.eval()
    predictions = []
    indices = []
    with torch.no_grad():
        for images, entity_idxs, _, _, idx in tqdm(test_loader):
            images = images.to(device)
            value_outputs, entity_outputs, unit_outputs = model(images)
            predicted_values = value_outputs.squeeze().cpu().numpy()
            predicted_entities = entity_outputs.argmax(dim=1).cpu().numpy()
            predicted_units = unit_outputs.argmax(dim=1).cpu().numpy()
            for value, entity_idx, unit_idx, sample_idx in zip(predicted_values, predicted_entities, predicted_units, idx):
                entity = idx_to_entity[entity_idx]
                unit = idx_to_unit[unit_idx]
                predictions.append(f"{value:.2f} {unit}")
                indices.append(sample_idx)
    return predictions, indices

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    train_df = pd.read_csv('/content/drive/MyDrive/cb lects/66e31d6ee96cd_student_resource_3/student_resource 3/dataset/train.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/cb lects/66e31d6ee96cd_student_resource_3/student_resource 3/dataset/test.csv')

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = ProductImageDataset('/content/drive/MyDrive/cb lects/66e31d6ee96cd_student_resource_3/student_resource 3/dataset/train.csv', 'train_images', transform=transform)
    test_dataset = ProductImageDataset('/content/drive/MyDrive/cb lects/66e31d6ee96cd_student_resource_3/student_resource 3/dataset/test.csv', 'test_images', transform=transform, is_test=True)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=1)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=1)


    num_entities = len(entity_unit_map)
    num_units = len(train_dataset.unit_to_idx)
    model = CRNN(num_entities, num_units).to(device)


    criterion_value = nn.MSELoss()
    criterion_entity = nn.CrossEntropyLoss()
    criterion_unit = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 10

    for epoch in range(num_epochs):
        train(model, train_loader, criterion_value, criterion_entity, criterion_unit, optimizer, device)
        val_loss = validate(model, train_loader, criterion_value, criterion_entity, criterion_unit, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

    idx_to_entity = {idx: entity for entity, idx in train_dataset.entity_to_idx.items()}
    idx_to_unit = {idx: unit for unit, idx in train_dataset.unit_to_idx.items()}
    predictions, indices = predict(model, test_loader, device, idx_to_entity, idx_to_unit)

    output_df = pd.DataFrame({
        'index': indices,
        'prediction': predictions
    })

    # Saving predictions
    output_file = 'test_out1.csv'
    output_df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")

if __name__ == "__main__":
    main()

 21%|██▏       | 1766/8246 [11:55:40<46:53:50, 26.05s/it]